YT video link: [Let's build GPT: from scratch, in code, spelled out](https://www.youtube.com/watch?v=kCc8FmEb1nY&ab_channel=AndrejKarpathy)

Need to decide:  
  

*   Train Data => 10,000
*   Test Data => 1,000



Vocab Size: 32 => a-z, “.”, “,”, “!”, END

Context Size: 4

Assignment: Train this pico GPT

Training Set: => 1000, 10K characters, maybe, the harry potter wikipedia page

Test Set: => 100, 1000 character

Output at end of the training process is a set of weight: N model weights

Inference code*:
Python N model weights => Do the prediction


In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
with open('/content/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [34]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

def preprocess_text(text, training_chars=10000, test_chars=1000):

    text = text.lower()

    allowed_chars = set("abcdefghijklmnopqrstuvwxyz.,!:? ")
    filtered_text = ''.join([ch for ch in text if ch in allowed_chars])

    training_text = filtered_text[:training_chars]
    test_text = filtered_text[training_chars:training_chars+test_chars]

    return training_text, test_text

training_text, test_text = preprocess_text(text)

len(training_text), len(test_text)

(10000, 1000)

In [4]:
print(training_text)
print(test_text)

mr. and mrs. dursley, of number four, privet drive, were proud to saythat they were perfectly normal, thank you very much. they were the lastpeople youd expect to be involved in anything strange or mysterious,because they just didnt hold with such nonsense.mr. dursley was the director of a firm called grunnings, which madedrills. he was a big, beefy man with hardly any neck, although he didhave a very large mustache. mrs. dursley was thin and blonde and hadnearly twice the usual amount of neck, which came in very useful as shespent so much of her time craning over garden fences, spying on theneighbors. the dursleys had a small son called dudley and in theiropinion there was no finer boy anywhere.the dursleys had everything they wanted, but they also had a secret, andtheir greatest fear was that somebody would discover it. they didntthink they could bear it if anyone found out about the potters. mrs.potter was mrs. dursleys sister, but they hadnt met for several yearsin fact, mrs. dursl

In [35]:
class TextDataset(Dataset):
    def __init__(self, text, seq_length):
        chars = sorted(list(set(text)))
        self.char_to_idx = {ch: i for i, ch in enumerate(chars)}
        self.idx_to_char = {i: ch for i, ch in enumerate(chars)}
        self.vocab_size = len(chars)
        self.data = [self.char_to_idx[ch] for ch in text]
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        return (torch.tensor(self.data[index:index+self.seq_length]),
                torch.tensor(self.data[index+1:index+self.seq_length+1]))

seq_length = 4  # Context size
batch_size = 32
dataset = TextDataset(training_text, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print(dataset.vocab_size)

13


In [6]:
sample_seq, sample_target = dataset[0]

print("\nSample Encoded Text and Target:")
print("Encoded Text:", sample_seq)
print("Encoded Target:", sample_target)



Sample Encoded Text and Target:
Encoded Text: tensor([18, 23,  3,  0])
Encoded Target: tensor([23,  3,  0,  6])


In [ ]:
# Print vocabulary
print("Vocabulary (Character to Index):")
for char, idx in dataset.char_to_idx.items():
    print(f"'{char}': {idx}")

Vocabulary (Character to Index):
' ': 0
'!': 1
',': 2
'.': 3
':': 4
'?': 5
'a': 6
'b': 7
'c': 8
'd': 9
'e': 10
'f': 11
'g': 12
'h': 13
'i': 14
'j': 15
'k': 16
'l': 17
'm': 18
'n': 19
'o': 20
'p': 21
'q': 22
'r': 23
's': 24
't': 25
'u': 26
'v': 27
'w': 28
'x': 29
'y': 30
'z': 31


In [ ]:
def decode_text(encoded_text, dataset):
    return ''.join(dataset.idx_to_char[idx] for idx in encoded_text)

# Example encoded text
encoded_text = [dataset.char_to_idx[ch] for ch in "harry"]
decoded_text = decode_text(encoded_text, dataset)

print("Encoded Text:", encoded_text)
print("Decoded Text:", decoded_text)

Encoded Text: [13, 6, 23, 23, 30]
Decoded Text: harry


In [36]:
class picoGPT(nn.Module):
    def __init__(self, vocab_size, seq_length, embedding_dim=1, num_heads=1, num_layers=1):
        super(picoGPT, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embedding_dim, vocab_size)
        self.seq_length = seq_length

    def forward(self, x):
        embedded = self.embed(x).permute(1, 0, 2)
        transformed = self.transformer_encoder(embedded)
        logits = self.fc_out(transformed.permute(1, 0, 2))
        return logits

model = picoGPT(vocab_size=dataset.vocab_size, seq_length=seq_length)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
sum(p.numel() for p in model.parameters() if p.requires_grad)

6196

In [ ]:
epochs = 100
for epoch in range(epochs):
    total_loss = 0
    for input, target in dataloader:
        optimizer.zero_grad()
        output = model(input)
        loss = loss_fn(output.view(-1, dataset.vocab_size), target.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

Epoch 1, Loss: 2.000478434562683
Epoch 2, Loss: 1.799840043258667
Epoch 3, Loss: 1.730271402168274
Epoch 4, Loss: 1.6937616897583008
Epoch 5, Loss: 1.6612870164871216
Epoch 6, Loss: 1.6286494258880615
Epoch 7, Loss: 1.5920538747787476
Epoch 8, Loss: 1.5590175336837768
Epoch 9, Loss: 1.5463937314987182
Epoch 10, Loss: 1.5325832685470582
Epoch 11, Loss: 1.5404346927642822
Epoch 12, Loss: 1.5373177854537965
Epoch 13, Loss: 1.5501397792816163
Epoch 14, Loss: 1.5023185979843139
Epoch 15, Loss: 1.497276449394226
Epoch 16, Loss: 1.4969662841796876
Epoch 17, Loss: 1.487474423980713
Epoch 18, Loss: 1.5062421339035035
Epoch 19, Loss: 1.545079416656494
Epoch 20, Loss: 1.546852135658264
Epoch 21, Loss: 1.5441715835571288
Epoch 22, Loss: 1.5139746381759644
Epoch 23, Loss: 1.5038782751083375
Epoch 24, Loss: 1.5047176809310914
Epoch 25, Loss: 1.5497445240020753
Epoch 26, Loss: 1.5311887134552002
Epoch 27, Loss: 1.5234568597793579
Epoch 28, Loss: 1.5260723907470704
Epoch 29, Loss: 1.489927777671814
Ep

In [ ]:
test_dataset = TextDataset(test_text, seq_length)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for input, target in dataloader:
            output = model(input)
            loss = loss_fn(output.view(-1, dataset.vocab_size), target.view(-1))
            total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    return avg_loss

test_loss = evaluate(model, test_dataloader)
print(f'Test Loss: {test_loss}')

Test Loss: 4.8439044044131325


In [ ]:
def predict(model, start_str, length=10):
    model.eval()
    input_seq = torch.tensor([dataset.char_to_idx[ch] for ch in start_str[-seq_length:]], dtype=torch.long).unsqueeze(1)
    predicted_text = start_str
    for _ in range(length):
        with torch.no_grad():
            output = model(input_seq)
            predicted_index = output[-1, :, :].argmax(dim=1).item()
            predicted_text += dataset.idx_to_char[predicted_index]
            input_seq = torch.cat((input_seq[1:], torch.tensor([[predicted_index]])), dim=0)
    return predicted_text


# Inference
print(predict(model, start_str="haryy"))

haryyyyyyyyyyyy
